In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable
from PIL import Image
import cv2
import albumentations as A
import time
import os
from tqdm.notebook import tqdm
from torchsummary import summary
import segmentation_models_pytorch as smp

!pip install -q torchsummary
!pip install -q segmentation-models-pytorch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files
from google.colab import drive

drive.mount("/content/drive")

dataset_path = os.path.join('/content/drive/My Drive', 'V4A/final_val_multi2')

print(dataset_path)


Mounted at /content/drive
/content/drive/My Drive/V4A/final_val_multi2


In [ ]:
IMAGE_PATH = '/content/drive/My Drive/V4A/final_val_multi2/rgb/'
MASK_PATH = '/content/drive/My Drive/V4A/final_val_multi2/Combined_Masks/'

In [ ]:
n_classes = 4

def create_df():
    name = []
    for dirname, _, filenames in os.walk(IMAGE_PATH):
        for filename in filenames:
            name.append(filename.split('.')[0])

    return pd.DataFrame({'id': name}, index = np.arange(0, len(name)))

df = create_df()
print('Total Images: ', len(df))

Total Images:  750


In [ ]:
X_train = []  # Empty list for training set since you're not using it
X_test = df['id'].values  # Assigning all samples to the test set

print('Train Size   : ', len(X_train))
print('Test Size    : ', len(X_test))


Train Size   :  0
Test Size    :  750


In [ ]:
class CustomDataset(Dataset):

    def __init__(self, img_path, mask_path, X,transform=None, patch=False):
        self.img_path = img_path
        self.mask_path = mask_path
        self.X = X
        self.transform = transform
        self.patches = patch
        # self.mean = mean
        # self.std = std

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img = cv2.imread(self.img_path + self.X[idx] + '.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_path + self.X[idx] + '.png', cv2.IMREAD_GRAYSCALE)

        if self.transform is not None:
            aug = self.transform(image=img, mask=mask)
            img = Image.fromarray(aug['image'])
            mask = aug['mask']

        if self.transform is None:
            img = Image.fromarray(img)

        t = T.Compose([T.ToTensor()])
        img = t(img)
        mask = torch.from_numpy(mask).long()

        if self.patches:
            img, mask = self.tiles(img, mask)

        return img, mask

    # def tiles(self, img, mask):

    #     img_patches = img.unfold(1, 512, 512).unfold(2, 768, 768)
    #     img_patches  = img_patches.contiguous().view(3,-1, 512, 768)
    #     img_patches = img_patches.permute(1,0,2,3)

    #     mask_patches = mask.unfold(0, 512, 512).unfold(1, 768, 768)
    #     mask_patches = mask_patches.contiguous().view(-1, 512, 768)

    #     return img_patches, mask_patches

In [ ]:
# mean = [0.485, 0.456, 0.406]
# std = [0.229, 0.224, 0.225]

t_test = None  # No transformations for testing

# Test dataset
test_set = CustomDataset(IMAGE_PATH, MASK_PATH, X_test,transform=t_test, patch=False)

# Test data loader
batch_size = 16
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)  # No need to shuffle for testing

In [ ]:
# import torch
# !pip install -q segmentation-models-pytorch
# !pip install -q torchsummary

# from torchsummary import summary
# import segmentation_models_pytorch as smp
# # Define the path to the saved model
# model_path = '/content/RJB_514_Unet-Mobilenetv2-transformations.pth'


# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = torch.load(model_path, map_location=device)

# # If the model was saved with DataParallel, you may need to unwrap it
# if isinstance(model, torch.nn.DataParallel):
#     model = model.module

# # Make sure to move the model to the appropriate device (CPU or GPU) if necessary
# model = model.to(device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.5 MB/s eta 0:00:00


In [ ]:
def pixel_accuracy(output, mask):
    with torch.no_grad():
        output = torch.argmax(F.softmax(output, dim=1), dim=1)
        correct = torch.eq(output, mask).int()
        accuracy = float(correct.sum()) / float(correct.numel())
    return accuracy

In [ ]:
# def mIoU(pred_mask, mask, smooth=1e-10, n_classes=4):
#     with torch.no_grad():
#         pred_mask = F.softmax(pred_mask, dim=1)
#         pred_mask = torch.argmax(pred_mask, dim=1)
#         pred_mask = pred_mask.contiguous().view(-1)
#         mask = mask.contiguous().view(-1)

#         iou_per_class = []
#         for clas in range(0, n_classes): #loop per pixel class
#             true_class = pred_mask == clas
#             true_label = mask == clas

#             if true_label.long().sum().item() == 0: #no exist label in this loop
#                 iou_per_class.append(np.nan)
#             else:
#                 intersect = torch.logical_and(true_class, true_label).sum().float().item()
#                 union = torch.logical_or(true_class, true_label).sum().float().item()

#                 iou = (intersect + smooth) / (union +smooth)
#                 iou_per_class.append(iou)
#         return np.nanmean(iou_per_class)

In [ ]:
import torch.nn.functional as F

def mIoU(pred_mask, mask, smooth=1e-10, n_classes=4):
    with torch.no_grad():
        pred_mask = F.softmax(pred_mask, dim=1)
        pred_mask = torch.argmax(pred_mask, dim=1)
        pred_mask = pred_mask.contiguous().view(-1)
        mask = mask.contiguous().view(-1)

        iou_per_class = []
        for clas in range(0, n_classes): #loop per pixel class
            true_class = pred_mask == clas
            true_label = mask == clas

            if true_label.long().sum().item() == 0: #no exist label in this loop
                iou_per_class.append(np.nan)
            else:
                intersect = torch.logical_and(true_class, true_label).sum().float().item()
                union = torch.logical_or(true_class, true_label).sum().float().item()

                iou = (intersect + smooth) / (union + smooth)
                iou_per_class.append(iou)
        mean_iou = np.nanmean(iou_per_class)
        return mean_iou, iou_per_class


In [ ]:
def fit(epochs, model, test_loader, criterion, optimizer, scheduler, patch=False):
    torch.cuda.empty_cache()
    test_losses = []
    val_iou = []
    val_acc = []
    lrs = []
    forward_times = []
    backward_times = []
    val_iou_per_class = [[] for _ in range(n_classes)]
    val_acc_per_class = [[] for _ in range(n_classes)]

    model.to(device)
    fit_time = time.time()

    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Turn off gradient computation for evaluation
        test_loss = 0
        test_accuracy = 0
        val_iou_score = 0
        overall_correct = 0
        overall_total = 0
        class_correct = [0] * n_classes
        class_total = [0] * n_classes

        # Loop over the test dataset
        for i, data in enumerate(tqdm(test_loader)):
            # Reshape to patches if necessary
            image_tiles, mask_tiles = data

            if patch:
                bs, n_tiles, c, h, w = image_tiles.size()

                image_tiles = image_tiles.view(-1, c, h, w)
                mask_tiles = mask_tiles.view(-1, h, w)

            image = image_tiles.to(device)
            mask = mask_tiles.to(device)
            output = model(image)

            # Evaluation metrics
            val_iou_score, val_iou_per_class_batch = mIoU(output, mask)
            val_iou.append(val_iou_score)
            for j in range(n_classes):
                val_iou_per_class[j].append(val_iou_per_class_batch[j])

            # Pixel accuracy per class
            pred_labels = torch.argmax(output, dim=1)
            correct_per_class = pred_labels.eq(mask).float()
            for j in range(n_classes):
                class_correct[j] += torch.sum(correct_per_class[mask == j])
                class_total[j] += torch.sum(mask == j)

            # Overall pixel accuracy
            overall_correct += torch.sum(pred_labels == mask)
            overall_total += mask.numel()

            # Loss
            loss = criterion(output, mask)
            test_loss += loss.item()

    # Calculate pixel accuracy overall
    overall_accuracy = overall_correct.item() / overall_total
    val_acc.append(overall_accuracy)

    # Calculate pixel accuracy per class
    for j in range(n_classes):
        if class_total[j] == 0:
            val_acc_per_class[j].append(0)
        else:
            val_acc_per_class[j].append(class_correct[j].item() / class_total[j])

    test_losses.append(test_loss / len(test_loader))
    val_iou.append(val_iou_score / len(test_loader))

    history = {'val_loss': test_losses,
               'val_miou': val_iou,
               'val_acc': val_acc,
               'val_miou_per_class': val_iou_per_class,
               'val_acc_per_class': val_acc_per_class,
               'forward_times': forward_times,
               'backward_times': backward_times
               }

    print('Total time: {:.2f} m'.format((time.time() - fit_time) / 60))
    return history


In [ ]:
# max_lr = 1e-3
# epoch = 15
# weight_decay = 1e-4

# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr=max_lr, weight_decay=weight_decay)
# sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epoch,
#                                             steps_per_epoch=len(test_loader))

# history = fit(epoch, model, test_loader, criterion, optimizer, sched)

  0%|          | 0/47 [00:00<?, ?it/s]

Total time: 5.19 m


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import time

# model_paths = ["/content/RJB_513_Unet-Mobilenetv2-resize-no-transform.pth", "/content/RJB_514_Unet-Mobilenetv2-no-resize-no-transform.pth", "/content/RJB_514_Unet-Mobilenetv2-transformations.pth", "/content/RJB_514_Unet-Mobilenetv2-transformations_no_transfer_learning.pth"]
model_paths = ["/content/RJB_513_Unet-Mobilenetv2-resize-no-transform.pth","/content/RJB_514_Unet-Mobilenetv2-transformations.pth", "/content/RJB_514_Unet-Mobilenetv2-transformations_no_transfer_learning.pth", "/content/RJB_517_DeepLabV3Plus-Mobilenetv2-transformations_yes_transfer_learning.pth"]
max_lr = 1e-3
epoch = 15
weight_decay = 1e-4

criterion = nn.CrossEntropyLoss()

for model_path in model_paths:
    # Load the model
    model = torch.load(model_path)

    # Initialize optimizer and scheduler for the model
    optimizer = torch.optim.AdamW(model.parameters(), lr=max_lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epoch, steps_per_epoch=len(test_loader))

    # Fit the model
    history = fit(epoch, model, test_loader, criterion, optimizer, sched)

    # Get metrics
    val_iou_per_class = history['val_miou_per_class']
    val_iou = history['val_miou']
    val_acc_per_class = history['val_acc_per_class']
    val_acc = history['val_acc']

    val_mean_iou_per_class = [np.nanmean(iou_class) for iou_class in val_iou_per_class]

    for i, mean_iou in enumerate(val_mean_iou_per_class):
        print("Validation Mean IoU for Class {} for model {}: {:.4f}".format(i, model_path, mean_iou))

    print("Mean IoU for test for model {}: {:.4f}".format(model_path, val_iou[0]))  # Access the first (and only) element of the list
    print("Overall Pixel Accuracy for model {}: {:.4f}".format(model_path, val_acc[0]))

    for i, acc_class in enumerate(val_acc_per_class):
        print("Pixel Accuracy for Class {} for model {}: {:.4f}".format(i, model_path, acc_class[0]))  # Access the first (and only) element of the list


100%|██████████| 47/47 [05:16<00:00,  6.74s/it]


Total time: 5.28 m
Validation Mean IoU for Class 0 for model /content/RJB_513_Unet-Mobilenetv2-resize-no-transform.pth: 0.6263
Validation Mean IoU for Class 1 for model /content/RJB_513_Unet-Mobilenetv2-resize-no-transform.pth: 0.0334
Validation Mean IoU for Class 2 for model /content/RJB_513_Unet-Mobilenetv2-resize-no-transform.pth: 0.0000
Validation Mean IoU for Class 3 for model /content/RJB_513_Unet-Mobilenetv2-resize-no-transform.pth: 0.0285
Mean IoU for test for model /content/RJB_513_Unet-Mobilenetv2-resize-no-transform.pth: 0.2867
Overall Pixel Accuracy for model /content/RJB_513_Unet-Mobilenetv2-resize-no-transform.pth: 0.6190
Pixel Accuracy for Class 0 for model /content/RJB_513_Unet-Mobilenetv2-resize-no-transform.pth: 0.9492
Pixel Accuracy for Class 1 for model /content/RJB_513_Unet-Mobilenetv2-resize-no-transform.pth: 0.0360
Pixel Accuracy for Class 2 for model /content/RJB_513_Unet-Mobilenetv2-resize-no-transform.pth: 0.0000
Pixel Accuracy for Class 3 for model /content/R

100%|██████████| 47/47 [00:19<00:00,  2.44it/s]


Total time: 0.32 m
Validation Mean IoU for Class 0 for model /content/RJB_514_Unet-Mobilenetv2-transformations.pth: 0.6439
Validation Mean IoU for Class 1 for model /content/RJB_514_Unet-Mobilenetv2-transformations.pth: 0.0213
Validation Mean IoU for Class 2 for model /content/RJB_514_Unet-Mobilenetv2-transformations.pth: 0.0000
Validation Mean IoU for Class 3 for model /content/RJB_514_Unet-Mobilenetv2-transformations.pth: 0.0089
Mean IoU for test for model /content/RJB_514_Unet-Mobilenetv2-transformations.pth: 0.3253
Overall Pixel Accuracy for model /content/RJB_514_Unet-Mobilenetv2-transformations.pth: 0.6443
Pixel Accuracy for Class 0 for model /content/RJB_514_Unet-Mobilenetv2-transformations.pth: 0.9980
Pixel Accuracy for Class 1 for model /content/RJB_514_Unet-Mobilenetv2-transformations.pth: 0.0080
Pixel Accuracy for Class 2 for model /content/RJB_514_Unet-Mobilenetv2-transformations.pth: 0.0000
Pixel Accuracy for Class 3 for model /content/RJB_514_Unet-Mobilenetv2-transformati

100%|██████████| 47/47 [00:19<00:00,  2.43it/s]


Total time: 0.32 m
Validation Mean IoU for Class 0 for model /content/RJB_514_Unet-Mobilenetv2-transformations_no_transfer_learning.pth: 0.6125
Validation Mean IoU for Class 1 for model /content/RJB_514_Unet-Mobilenetv2-transformations_no_transfer_learning.pth: 0.1323
Validation Mean IoU for Class 2 for model /content/RJB_514_Unet-Mobilenetv2-transformations_no_transfer_learning.pth: 0.0000
Validation Mean IoU for Class 3 for model /content/RJB_514_Unet-Mobilenetv2-transformations_no_transfer_learning.pth: 0.0591
Mean IoU for test for model /content/RJB_514_Unet-Mobilenetv2-transformations_no_transfer_learning.pth: 0.3065
Overall Pixel Accuracy for model /content/RJB_514_Unet-Mobilenetv2-transformations_no_transfer_learning.pth: 0.6217
Pixel Accuracy for Class 0 for model /content/RJB_514_Unet-Mobilenetv2-transformations_no_transfer_learning.pth: 0.9180
Pixel Accuracy for Class 1 for model /content/RJB_514_Unet-Mobilenetv2-transformations_no_transfer_learning.pth: 0.1580
Pixel Accuracy

100%|██████████| 47/47 [00:18<00:00,  2.61it/s]

Total time: 0.30 m
Validation Mean IoU for Class 0 for model /content/RJB_517_DeepLabV3Plus-Mobilenetv2-transformations_yes_transfer_learning.pth: 0.6068
Validation Mean IoU for Class 1 for model /content/RJB_517_DeepLabV3Plus-Mobilenetv2-transformations_yes_transfer_learning.pth: 0.1391
Validation Mean IoU for Class 2 for model /content/RJB_517_DeepLabV3Plus-Mobilenetv2-transformations_yes_transfer_learning.pth: 0.0000
Validation Mean IoU for Class 3 for model /content/RJB_517_DeepLabV3Plus-Mobilenetv2-transformations_yes_transfer_learning.pth: 0.1547
Mean IoU for test for model /content/RJB_517_DeepLabV3Plus-Mobilenetv2-transformations_yes_transfer_learning.pth: 0.3216
Overall Pixel Accuracy for model /content/RJB_517_DeepLabV3Plus-Mobilenetv2-transformations_yes_transfer_learning.pth: 0.6192
Pixel Accuracy for Class 0 for model /content/RJB_517_DeepLabV3Plus-Mobilenetv2-transformations_yes_transfer_learning.pth: 0.8894
Pixel Accuracy for Class 1 for model /content/RJB_517_DeepLabV3P

In [ ]:
val_iou_per_class = history['val_miou_per_class']
val_iou = history['val_miou']
val_acc_per_class = history['val_acc_per_class']
val_acc = history['val_acc']

val_mean_iou_per_class = [np.nanmean(iou_class) for iou_class in val_iou_per_class]

for i, mean_iou in enumerate(val_mean_iou_per_class):
    print("Validation Mean IoU for Class {}: {:.4f}".format(i, mean_iou))

print("Mean IoU for test: {:.4f}".format(val_iou[0]))  # Access the first (and only) element of the list
print("Overall Pixel Accuracy: {:.4f}".format(val_acc[0]))

for i, acc_class in enumerate(val_acc_per_class):
    print("Pixel Accuracy for Class {}: {:.4f}".format(i, acc_class[0]))  # Access the first (and only) element of the list


Validation Mean IoU for Class 0: 0.6439
Validation Mean IoU for Class 1: 0.0213
Validation Mean IoU for Class 2: 0.0000
Validation Mean IoU for Class 3: 0.0089
Mean IoU for test: 0.3253
Overall Pixel Accuracy: 0.6443
Pixel Accuracy for Class 0: 0.9980
Pixel Accuracy for Class 1: 0.0080
Pixel Accuracy for Class 2: 0.0000
Pixel Accuracy for Class 3: 0.0097


In [ ]:
# import torch.nn.functional as F

# def mIoU(pred_mask, mask, smooth=1e-10, n_classes=4):
#     with torch.no_grad():
#         pred_mask = F.softmax(pred_mask, dim=1)
#         pred_mask = torch.argmax(pred_mask, dim=1)
#         pred_mask = pred_mask.contiguous().view(-1)
#         mask = mask.contiguous().view(-1)

#         iou_per_class = []
#         for clas in range(0, n_classes): # loop per pixel class
#             true_class = pred_mask == clas
#             true_label = mask == clas

#             intersect = torch.logical_and(true_class, true_label).sum().float().item()
#             union = torch.logical_or(true_class, true_label).sum().float().item()

#             iou = (intersect + smooth) / (union + smooth)
#             iou_per_class.append(iou)

#         mean_iou = np.nanmean(iou_per_class)
#         return mean_iou, iou_per_class

# import random

# # Define class colors and titles
# class_colors = {
#     0: [91, 6, 117],  # Background purple
#     1: [255, 0, 0],   # Dry down is red
#     2: [0, 0, 255],   # Nutrient deficiency is blue
#     3: [0, 153, 0],     # Weed cluster is black
#     # Add more colors for additional classes if needed
# }

# class_titles = {
#     0: 'Background',
#     1: 'Dry down',
#     2: 'Nutrient deficiency',
#     3: 'Weed cluster',
#     # Add titles for additional classes if needed
# }

# # Randomly select 10 indices from the test set
# selected_indices = list(range(1, 11)) + list(range(50, 61)) + list(range(100, 111)) + list(range(150, 161)) + list(range(200, 211)) + list(range(285, 296))

# # Plotting for each randomly selected image
# for idx in selected_indices:
#     image, mask = test_set[idx]

#     # Convert image and mask to PyTorch tensors
#     image = image.unsqueeze(0).to(device)  # Add batch dimension and move to device
#     mask = mask.unsqueeze(0).to(device)  # Add batch dimension and move to device

#     # Forward pass through the model to get the predicted mask
#     with torch.no_grad():
#         model.eval()
#         output = model(image)  # Forward pass
#         pred_mask = torch.argmax(output, dim=1).squeeze().cpu()  # Convert tensor to CPU and remove batch dimension

#     # Convert predicted mask to RGB image with class colors
#     pred_mask_rgb = torch.zeros((3, pred_mask.shape[0], pred_mask.shape[1]), dtype=torch.uint8)
#     for class_idx, color in class_colors.items():
#         pred_mask_rgb[:, pred_mask == class_idx] = torch.tensor(color, dtype=torch.uint8).view(3, 1)

#     # Calculate Intersection over Union (IoU)
#     iou, iou_per_class = mIoU(output, mask)

#     # Plotting
#     fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 10))
#     ax1.imshow(image.squeeze().permute(1, 2, 0).cpu().numpy())
#     ax1.set_title('Picture')
#     ax1.axis('off')

#     ax2.imshow(mask.squeeze().cpu().numpy())
#     ax2.set_title('Ground truth')
#     ax2.axis('off')

#     ax3.imshow(pred_mask_rgb.permute(1, 2, 0).cpu().numpy())
#     ax3.set_title(f'UNet-MobileNetv2 (Predicted)\nIoU: {iou:.4f}')  # Include IoU in the title
#     ax3.axis('off')

#     # Add legend for class colors with class titles
#     handles = [plt.Rectangle((0, 0), 1, 1, color=np.array(color) / 255) for color in class_colors.values()]
#     labels = [class_titles[class_idx] for class_idx in class_colors.keys()]
#     ax3.legend(handles, labels)
#     fig.patch.set_alpha(0)
#     plt.show()


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

def mIoU(pred_mask, mask, smooth=1e-10, n_classes=4):
    with torch.no_grad():
        pred_mask = F.softmax(pred_mask, dim=1)
        pred_mask = torch.argmax(pred_mask, dim=1)
        pred_mask = pred_mask.contiguous().view(-1)
        mask = mask.contiguous().view(-1)

        iou_per_class = []
        for clas in range(0, n_classes): # loop per pixel class
            true_class = pred_mask == clas
            true_label = mask == clas

            intersect = torch.logical_and(true_class, true_label).sum().float().item()
            union = torch.logical_or(true_class, true_label).sum().float().item()

            iou = (intersect + smooth) / (union + smooth)
            iou_per_class.append(iou)

        mean_iou = np.nanmean(iou_per_class)
        return mean_iou, iou_per_class

# Define class colors and titles
class_colors = {
    0: [91, 6, 117],  # Background purple
    1: [255, 0, 0],   # Dry down is red
    2: [0, 0, 255],   # Nutrient deficiency is blue
    3: [0, 153, 0],   # Weed cluster is green
}

class_titles = {
    0: 'Background',
    1: 'Dry down',
    2: 'Nutrient deficiency',
    3: 'Weed cluster',
}

# Load the models
model_paths = ["/content/RJB_513_Unet-Mobilenetv2-resize-no-transform.pth","/content/RJB_514_Unet-Mobilenetv2-transformations.pth", "/content/RJB_514_Unet-Mobilenetv2-transformations_no_transfer_learning.pth", "/content/RJB_517_DeepLabV3Plus-Mobilenetv2-transformations_yes_transfer_learning.pth"]
models = []
for path in model_paths:
    model = torch.load(path)
    model.eval()
    models.append(model)

# Randomly select indices from the test set
selected_indices = list(range(1, 55)) + list(range(150, 200)) + list(range(250, 300)) + list(range(450, 500)) + list(range(545, 600)) + list(range(700, 725))

# Plotting for each randomly selected image
for idx in selected_indices:
    image, mask = test_set[idx]

    # Convert image and mask to PyTorch tensors
    image = image.unsqueeze(0).to(device)  # Add batch dimension and move to device
    mask = mask.unsqueeze(0).to(device)  # Add batch dimension and move to device

    # Check the dimensions of the mask
    if len(mask.shape) == 3:
        _, height, width = mask.shape
    elif len(mask.shape) == 4:
        _, _, height, width = mask.shape
    else:
        raise ValueError("Unexpected mask dimensions")

    fig, axes = plt.subplots(1, len(models) + 2, figsize=(18, 4))  # Reduced figure size

    ax1, ax2, *model_axes = axes

    # Display the original image
    ax1.imshow(image.squeeze().permute(1, 2, 0).cpu().numpy())
    ax1.set_title('Image')
    ax1.axis('off')

    # Convert ground truth mask to RGB image with class colors
    mask_rgb = torch.zeros((3, height, width), dtype=torch.uint8)
    for class_idx, color in class_colors.items():
        mask_rgb[:, mask.squeeze(0) == class_idx] = torch.tensor(color, dtype=torch.uint8).view(3, 1)

    # Display the ground truth mask
    ax2.imshow(mask_rgb.permute(1, 2, 0).cpu().numpy())
    ax2.set_title('Ground truth')
    ax2.axis('off')

    for model, ax in zip(models, model_axes):
        # Forward pass through the model to get the predicted mask
        with torch.no_grad():
            output = model(image)  # Forward pass
            pred_mask = torch.argmax(output, dim=1).squeeze().cpu()  # Convert tensor to CPU and remove batch dimension

        # Convert predicted mask to RGB image with class colors
        pred_mask_rgb = torch.zeros((3, pred_mask.shape[0], pred_mask.shape[1]), dtype=torch.uint8)
        for class_idx, color in class_colors.items():
            pred_mask_rgb[:, pred_mask == class_idx] = torch.tensor(color, dtype=torch.uint8).view(3, 1)

        # Calculate Intersection over Union (IoU)
        iou, iou_per_class = mIoU(output, mask)

        # Plot the predicted mask
        ax.imshow(pred_mask_rgb.permute(1, 2, 0).cpu().numpy())
        ax.set_title(f'Model {models.index(model) + 1}\nIoU: {iou:.4f}')  # Include IoU in the title
        ax.axis('off')

    # Adjust layout to make room for the legend
    plt.tight_layout(pad=0.1)
    plt.subplots_adjust(left=0.02, right=0.98, top=0.95, bottom=0.15, wspace=0.05)

    plt.show()

In [ ]:
# translucent background

import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

def mIoU(pred_mask, mask, smooth=1e-10, n_classes=4):
    with torch.no_grad():
        pred_mask = F.softmax(pred_mask, dim=1)
        pred_mask = torch.argmax(pred_mask, dim=1)
        pred_mask = pred_mask.contiguous().view(-1)
        mask = mask.contiguous().view(-1)

        iou_per_class = []
        for clas in range(0, n_classes):  # loop per pixel class
            true_class = pred_mask == clas
            true_label = mask == clas

            intersect = torch.logical_and(true_class, true_label).sum().float().item()
            union = torch.logical_or(true_class, true_label).sum().float().item()

            iou = (intersect + smooth) / (union + smooth)
            iou_per_class.append(iou)

        mean_iou = np.nanmean(iou_per_class)
        return mean_iou, iou_per_class

# Define class colors and titles
class_colors = {
    0: [91, 6, 117],  # Background purple
    1: [255, 0, 0],   # Dry down is red
    2: [0, 0, 255],   # Nutrient deficiency is blue
    3: [0, 153, 0],   # Weed cluster is green
}

class_titles = {
    0: 'Background',
    1: 'Dry down',
    2: 'Nutrient deficiency',
    3: 'Weed cluster',
}

# Load the models
model_paths = ["/content/RJB_513_Unet-Mobilenetv2-resize-no-transform.pth","/content/RJB_514_Unet-Mobilenetv2-transformations.pth", "/content/RJB_514_Unet-Mobilenetv2-transformations_no_transfer_learning.pth", "/content/RJB_517_DeepLabV3Plus-Mobilenetv2-transformations_yes_transfer_learning.pth"]
models = []
for path in model_paths:
    model = torch.load(path)
    model.eval()
    models.append(model)

# Randomly select indices from the test set
selected_indices = list(range(1, 55)) + list(range(150, 200)) + list(range(250, 300)) + list(range(450, 500)) + list(range(545, 600)) + list(range(700, 725))

# Plotting for each randomly selected image
for idx in selected_indices:
    image, mask = test_set[idx]

    # Convert image and mask to PyTorch tensors
    image = image.unsqueeze(0).to(device)  # Add batch dimension and move to device
    mask = mask.unsqueeze(0).to(device)  # Add batch dimension and move to device

    # Check the dimensions of the mask
    if len(mask.shape) == 3:
        _, height, width = mask.shape
    elif len(mask.shape) == 4:
        _, _, height, width = mask.shape
    else:
        raise ValueError("Unexpected mask dimensions")

    fig, axes = plt.subplots(1, len(models) + 2, figsize=(18, 4))  # Reduced figure size

    # Set the background of the figure to be translucent
    fig.patch.set_alpha(0)
    for ax in axes:
        ax.patch.set_alpha(0)

    ax1, ax2, *model_axes = axes

    # Display the original image
    ax1.imshow(image.squeeze().permute(1, 2, 0).cpu().numpy())
    ax1.set_title('Image')
    ax1.axis('off')

    # Convert ground truth mask to RGB image with class colors
    mask_rgb = torch.zeros((3, height, width), dtype=torch.uint8)
    for class_idx, color in class_colors.items():
        mask_rgb[:, mask.squeeze(0) == class_idx] = torch.tensor(color, dtype=torch.uint8).view(3, 1)

    # Display the ground truth mask
    ax2.imshow(mask_rgb.permute(1, 2, 0).cpu().numpy())
    ax2.set_title('Ground truth')
    ax2.axis('off')

    for model, ax in zip(models, model_axes):
        # Forward pass through the model to get the predicted mask
        with torch.no_grad():
            output = model(image)  # Forward pass
            pred_mask = torch.argmax(output, dim=1).squeeze().cpu()  # Convert tensor to CPU and remove batch dimension

        # Convert predicted mask to RGB image with class colors
        pred_mask_rgb = torch.zeros((3, pred_mask.shape[0], pred_mask.shape[1]), dtype=torch.uint8)
        for class_idx, color in class_colors.items():
            pred_mask_rgb[:, pred_mask == class_idx] = torch.tensor(color, dtype=torch.uint8).view(3, 1)

        # Calculate Intersection over Union (IoU)
        iou, iou_per_class = mIoU(output, mask)

        # Plot the predicted mask
        ax.imshow(pred_mask_rgb.permute(1, 2, 0).cpu().numpy())
        ax.set_title(f'Model {models.index(model) + 1}\nIoU: {iou:.4f}')  # Include IoU in the title
        ax.axis('off')

    # Adjust layout to make room for the legend
    plt.tight_layout(pad=0.1)
    plt.subplots_adjust(left=0.02, right=0.98, top=0.95, bottom=0.15, wspace=0.05)

    plt.show()


In [ ]:
# translucent background and vertical

import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

def mIoU(pred_mask, mask, smooth=1e-10, n_classes=4):
    with torch.no_grad():
        pred_mask = F.softmax(pred_mask, dim=1)
        pred_mask = torch.argmax(pred_mask, dim=1)
        pred_mask = pred_mask.contiguous().view(-1)
        mask = mask.contiguous().view(-1)

        iou_per_class = []
        for clas in range(0, n_classes):  # loop per pixel class
            true_class = pred_mask == clas
            true_label = mask == clas

            intersect = torch.logical_and(true_class, true_label).sum().float().item()
            union = torch.logical_or(true_class, true_label).sum().float().item()

            iou = (intersect + smooth) / (union + smooth)
            iou_per_class.append(iou)

        mean_iou = np.nanmean(iou_per_class)
        return mean_iou, iou_per_class

# Define class colors and titles
class_colors = {
    0: [91, 6, 117],  # Background purple
    1: [255, 0, 0],   # Dry down is red
    2: [0, 0, 255],   # Nutrient deficiency is blue
    3: [0, 153, 0],   # Weed cluster is green
}

class_titles = {
    0: 'Background',
    1: 'Dry down',
    2: 'Nutrient deficiency',
    3: 'Weed cluster',
}

# Load the models
model_paths = ["/content/RJB_513_Unet-Mobilenetv2-resize-no-transform.pth","/content/RJB_514_Unet-Mobilenetv2-transformations.pth", "/content/RJB_514_Unet-Mobilenetv2-transformations_no_transfer_learning.pth", "/content/RJB_517_DeepLabV3Plus-Mobilenetv2-transformations_yes_transfer_learning.pth"]
models = []
for path in model_paths:
    model = torch.load(path)
    model.eval()
    models.append(model)

# Randomly select indices from the test set
selected_indices = list(range(1, 55)) + list(range(150, 200)) + list(range(250, 300)) + list(range(450, 500)) + list(range(545, 600)) + list(range(700, 725))

# Plotting for each randomly selected image
for idx in selected_indices:
    image, mask = test_set[idx]

    # Convert image and mask to PyTorch tensors
    image = image.unsqueeze(0).to(device)  # Add batch dimension and move to device
    mask = mask.unsqueeze(0).to(device)  # Add batch dimension and move to device

    # Check the dimensions of the mask
    if len(mask.shape) == 3:
        _, height, width = mask.shape
    elif len(mask.shape) == 4:
        _, _, height, width = mask.shape
    else:
        raise ValueError("Unexpected mask dimensions")

    # Change 1: Update subplot layout to be vertical
    fig, axes = plt.subplots(len(models) + 2, 1, figsize=(5, 18))  # Changed figure size and layout to vertical

    # Set the background of the figure to be translucent
    fig.patch.set_alpha(0)
    for ax in axes:
        ax.patch.set_alpha(0)

    ax1, ax2, *model_axes = axes

    # Display the original image
    ax1.imshow(image.squeeze().permute(1, 2, 0).cpu().numpy())
    ax1.set_title('Image')
    ax1.axis('off')

    # Convert ground truth mask to RGB image with class colors
    mask_rgb = torch.zeros((3, height, width), dtype=torch.uint8)
    for class_idx, color in class_colors.items():
        mask_rgb[:, mask.squeeze(0) == class_idx] = torch.tensor(color, dtype=torch.uint8).view(3, 1)

    # Display the ground truth mask
    ax2.imshow(mask_rgb.permute(1, 2, 0).cpu().numpy())
    ax2.set_title('Ground truth')
    ax2.axis('off')

    for model, ax in zip(models, model_axes):
        # Forward pass through the model to get the predicted mask
        with torch.no_grad():
            output = model(image)  # Forward pass
            pred_mask = torch.argmax(output, dim=1).squeeze().cpu()  # Convert tensor to CPU and remove batch dimension

        # Convert predicted mask to RGB image with class colors
        pred_mask_rgb = torch.zeros((3, pred_mask.shape[0], pred_mask.shape[1]), dtype=torch.uint8)
        for class_idx, color in class_colors.items():
            pred_mask_rgb[:, pred_mask == class_idx] = torch.tensor(color, dtype=torch.uint8).view(3, 1)

        # Calculate Intersection over Union (IoU)
        iou, iou_per_class = mIoU(output, mask)

        # Plot the predicted mask
        ax.imshow(pred_mask_rgb.permute(1, 2, 0).cpu().numpy())
        ax.set_title(f'Model {models.index(model) + 1}\nIoU: {iou:.4f}')  # Include IoU in the title
        ax.axis('off')

    # Change 2: Adjust layout to fit the new vertical arrangement
    plt.tight_layout(pad=0.1)
    plt.subplots_adjust(left=0.1, right=0.9, top=0.95, bottom=0.05, hspace=0.5)

    plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
import random  # Add this line to import the random module

# Randomly select indices from the test set
selected_indices = random.sample(range(len(test_set)), 75)

# Plot each randomly selected image individually
for i, idx in enumerate(selected_indices):
    image, _ = test_set[idx]  # Ignore the mask

    # Convert image to numpy array and plot
    plt.figure(figsize=(4, 4))
    plt.imshow(image.permute(1, 2, 0).cpu().numpy())

    # Set the background of the figure and axes to be completely transparent
    plt.gca().set_facecolor('none')
    plt.gcf().patch.set_alpha(0)
    plt.gca().patch.set_alpha(0)

    plt.axis('off')
    plt.show()
